In [ ]:
import pandas as pd
import numpy as np
from tqdm import trange

In [ ]:
test_gold_path = '../data/test_gd/test.gold.en-de.json'
prediction_path = './final_results/en-de_0-shot.json'
test_gold_df = pd.read_json(test_gold_path)
prediction_df = pd.read_json(prediction_path)

Comparing acc directly by comparing string

In [ ]:
acc = 0.0
type_acc = [0.0, 0.0, 0.0, 0.0]
type_len = [0.0, 0.0, 0.0, 0.0]
type_name = ['prefix', 'suffix', 'bi_context', 'zero_context']
for i in range(len(test_gold_df)):
    if test_gold_df['context_type'][i] == 'prefix':
        if test_gold_df['target'][i].lower() == prediction_df['prediction'][i].lower():
            type_acc[0] += 1.0
        type_len[0] += 1.0
    elif test_gold_df['context_type'][i] == 'suffix':
        if test_gold_df['target'][i].lower() == prediction_df['prediction'][i].lower():
            type_acc[1] += 1.0
        type_len[1] += 1.0
    elif test_gold_df['context_type'][i] == 'bi_context':
        if test_gold_df['target'][i].lower() == prediction_df['prediction'][i].lower():
            type_acc[2] += 1.0
        type_len[2] += 1.0
    else:
        if test_gold_df['target'][i].lower() == prediction_df['prediction'][i].lower():
            type_acc[3] += 1.0
        type_len[3] += 1.0
    if test_gold_df['target'][i].lower() == prediction_df['prediction'][i].lower():
        acc += 1.0
print("The acc is {}".format(acc/len(test_gold_df)))
for i in range(len(type_acc)):
    print("The acc of {} is {}".format(type_name[i], type_acc[i]/type_len[i]))

Compute acc with lemmatizer

In [ ]:
import spacy
nlp = spacy.load("en_core_web_trf")
#nlp = spacy.load("de_dep_news_trf")
lemmatizer = nlp.get_pipe("lemmatizer")

In [ ]:
acc = 0.0
for i in trange(len(test_gold_df)):
    if prediction_df['prediction'][i] == '':
        continue
    temp1 = [token.lemma_ for token in nlp(test_gold_df['target'][i].lower())][0] 
    temp2 = [token.lemma_ for token in nlp(prediction_df['prediction'][i].lower())][0]
    if temp1 == temp2:
        acc += 1.0
print(acc/len(test_gold_df))

Compute the amount of predictions that don't start with the typed sequence 

In [ ]:
fault1_rate = 0.0
for i in trange(len(test_gold_df)):
    if not prediction_df['prediction'][i].lower().startswith(prediction_df['typed_seq'][i].lower()):
        fault1_rate+=1.0
print("The error rate of predictions don't start with the typed sequence is {}".format(fault1_rate/len(test_gold_df)))

Compute the amount of generation that contains a sequence of words

In [ ]:
language_type = 'de-en'
shot = 0
raw_prediction_df = pd.read_json('./results/inference_{}_{}-shot.json'.format(language_type, shot))
fault2_rate = 0.0
for i in range(len(raw_prediction_df)):
    generation = raw_prediction_df['generation'][i]
    str_list = generation.split('Answer: ')
    if len(str_list)>1:
        str_list = str_list[1].split(' ')
    else:
        str_list = str_list[0].split(' ')
    if len(str_list)>1:
        fault2_rate+=1.0
print(fault2_rate/len(raw_prediction_df))